In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from threading import Thread
import time

class IBApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.account_values = {}
        self.positions = []
        self.account_summary_finished = False
        self.positions_finished = False
        self.disconnected = False  # Flag to protect multiple disconnects

    def nextValidId(self, orderId):
        print(f"Connected. Next valid order id: {orderId}")
        self.reqAccountSummary(9001, "All", "TotalCashValue,AvailableFunds,BuyingPower,NetLiquidation")
        self.reqPositions()

    def accountSummary(self, reqId, account, tag, value, currency):
        print(f"Account Summary: Account: {account}, {tag} = {value} {currency}")
        self.account_values[tag] = (value, currency)

    def accountSummaryEnd(self, reqId):
        # Avoid printing multiple times for the same reqId if triggered repeatedly
        if not self.account_summary_finished:
            print("Finished account summary request.")
            self.account_summary_finished = True
            self.try_disconnect()

    def position(self, account, contract, position, avgCost):
        print(f"Position: {contract.symbol} | Qty: {position} | Avg Cost: {avgCost}")
        self.positions.append((account, contract, position, avgCost))

    def positionEnd(self):
        if not self.positions_finished:
            print("Finished positions request.")
            self.positions_finished = True
            self.try_disconnect()

    def try_disconnect(self):
        if self.account_summary_finished and self.positions_finished and not self.disconnected:
            print("Received all data, disconnecting.")
            self.disconnected = True
            self.disconnect()

    def error(self, reqId, errorCode, errorString):
        print(f"Error {errorCode}: {errorString}")


In [4]:
def run_loop(app):
    app.run()

app = IBApp()
app.connect("127.0.0.1", 7497, clientId=1) #7496 = Live, 7497 = Simulated Account 

api_thread = Thread(target=run_loop, args=(app,))
api_thread.start()

api_thread.join()


Connected. Next valid order id: 1
Error 2104: Market data farm connection is OK:usfarm.nj
Error 2104: Market data farm connection is OK:cashfarm
Error 2104: Market data farm connection is OK:usfarm
Error 2106: HMDS data farm connection is OK:euhmds
Error 2106: HMDS data farm connection is OK:fundfarm
Error 2106: HMDS data farm connection is OK:ushmds
Error 2158: Sec-def data farm connection is OK:secdefil
Finished positions request.
Account Summary: Account: DU7864794, AvailableFunds = 1080402.51 GBP
Account Summary: Account: DU7864794, BuyingPower = 4321610.04 GBP
Account Summary: Account: DU7864794, NetLiquidation = 1083840.84 GBP
Account Summary: Account: DU7864794, TotalCashValue = 1080402.51 GBP
Finished account summary request.
Received all data, disconnecting.
